In [1]:
from model import fastText
from utils import *
from model import *
import numpy as np
import sys
import torch.optim as optim
from torch import nn
import torch
from sklearn.metrics import accuracy_score, classification_report

In [2]:
train_file = '/Volumes/data/Data Science/Diploma/Detect-emotion-sentimental/dataset/booking/booking-sentences-train.csv'
test_file = '/Volumes/data/Data Science/Diploma/Detect-emotion-sentimental/dataset/booking/booking-sentences-test.csv'
val_file = '/Volumes/data/Data Science/Diploma/Detect-emotion-sentimental/dataset/booking/booking-sentences-val.csv'
w2v_file = '../data/ubercorpus.cased.tokenized.word2vec.300d'

In [3]:
class Config(object):
    embed_size = 300
    hidden_size = 10
    output_size = 2
    max_epochs = 30
    lr = 0.5
    batch_size = 128

In [4]:
config = Config()

In [5]:
dataset = Dataset(config)
dataset.load_data(w2v_file, train_file, test_file)

100%|█████████▉| 594235/595119 [00:59<00:00, 10587.22it/s]

Loaded 225090 training examples
Loaded 119849 test examples
Loaded 56273 validation examples


100%|█████████▉| 594235/595119 [01:10<00:00, 10587.22it/s]

 Create Model with specified optimizer and loss function

In [ ]:
model = fastText(config, len(dataset.vocab), dataset.word_embeddings)
if torch.cuda.is_available():
    model.cuda()
model.train()
optimizer = optim.SGD(model.parameters(), lr=config.lr)
NLLLoss = nn.NLLLoss()
model.add_optimizer(optimizer)
model.add_loss_op(NLLLoss)

In [ ]:
train_losses = []
val_accuracies = []

In [ ]:
def train():
    for i in range(config.max_epochs):
        print ("Epoch: {}".format(i))
        train_loss,val_accuracy = model.run_epoch(dataset.train_iterator, dataset.val_iterator, i)
        train_losses.append(train_loss)
        val_accuracies.append(val_accuracy)


In [ ]:
%time train()

In [ ]:
def calc_true_and_pred(model, iterator):
    all_preds = []
    all_y = []
    for idx,batch in enumerate(iterator):
        if torch.cuda.is_available():
            x = batch.text.cuda()
        else:
            x = batch.text
        y_pred = model(x)
        predicted = torch.max(y_pred.cpu().data, 1)[1] + 1
        all_preds.extend(predicted.numpy())
        all_y.extend(batch.label.numpy())
    return all_y, all_preds

In [ ]:
all_y, all_preds = calc_true_and_pred(model, dataset.test_iterator)

### accuracy score

In [ ]:
test_acc = accuracy_score(all_y, np.array(all_preds).flatten())
print ('Final Test Accuracy: {:.4f}'.format(test_acc))

### confusion matrix

In [ ]:
report = classification_report(all_y, np.array(all_preds).flatten())

In [ ]:
print(report)